In [1]:
import numpy as np
import torch
from sklearn import ensemble, metrics
import pandas as pd
import csv
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from lib import model_device_io as io

In [2]:
def GetDateLabel(ls):
    assert(len(ls) == 3)
    output = ''
    for mem in ls:
        output += str(mem)+'/'
    return output[:-1]

def GetDateLabelCvt(ip):
    num_list = [int(mem) for mem in ip.split('-')]
    return GetDateLabel(num_list)
    
class HotelBooking(Dataset):
    def __init__(self, fn, fn_label):
        data = pd.read_csv(fn)
        data_len = len(data['arrival_date_year'])
        self.vector_list = []
        for idx in range(data_len):
            vec = data.iloc[idx]
            vec = [key for key in vec]
            vec = torch.tensor(vec)
            self.vector_list.append(vec[1:])
            
        data_label = pd.read_csv(fn_label)
        data_len = len(data['arrival_date_year'])
        self.label_list = []
        for idx in range(data_len):
            label = data_label['adr'][idx]
            self.label_list.append(label)
        
        self.len = len(self.label_list)
        
    def __getitem__(self, idx):
        return self.vector_list[idx], torch.tensor(self.label_list[idx])
    
    def __len__(self):
        return self.len

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hid1 = nn.Linear(245, 100)  # 13-(10-10)-1
        self.hid2 = nn.Linear(100, 50)
        self.oupt = nn.Linear(50, 1)
        
    def forward(self, x):
        z = torch.tanh(self.hid1(x))
        z = torch.tanh(self.hid2(z))
        z = self.oupt(z)
        return z

In [8]:
def test():
    model.eval()
    loss = 0
    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(test_loader):
            data, label = data.cuda().float(), label.cuda().float()
            output = model(data)
            loss += loss_func(output, label).item()*len(data)
        loss /= len(test_loader.dataset)
        print('***Validation:\tAccuracy: {:.4f}'.format(loss.item()),'\n')
    return loss

def train(n_epoch = 100):
    min_loss = 9999999999999999
    for epoch in range(n_epoch):
        model.train()
        for batch_idx, (data, label) in enumerate(train_loader):
            model.zero_grad()
            data, label = data.cuda().float(), label.cuda().float()
            output = model(data)
            loss = loss_func(output, label)
            loss.backward()
            optimizer.step()
            if batch_idx % max(1, int(len(train_loader)/1)) == 0:
                print('Epoch: {}/{}\tProgress: {:.2f}% ({}/{})\tLoss: {:.4f}'.format(epoch+1, n_epoch,
                      (batch_idx+1)*100./len(train_loader), batch_idx+1, len(train_loader), loss.item()))
                      
        #new_loss = test()
       #if new_loss < min_loss:
            #min_loss = new_loss
        io.saveModel('models/best.pth', model, optimizer)

In [12]:
loss_func = nn.MSELoss()
model = Net().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0004, betas=(0.5, 0.999))
train_set = HotelBooking('data/preprocessed/train_processed.csv', 'data/preprocessed/adr.csv')
io.loadModel('models/best.pth', model, optimizer)
train_loader = DataLoader(train_set, batch_size=4096, shuffle=True, num_workers=0)

#io.loadModel('models/best.pth', model, optimizer)
test_loader = train_set

model loaded from models/best.pth


In [13]:
train()

Epoch: 1/100	Progress: 4.35% (1/23)	Loss: 3298.4170
model saved to models/best.pth
Epoch: 2/100	Progress: 4.35% (1/23)	Loss: 3241.9614
model saved to models/best.pth
Epoch: 3/100	Progress: 4.35% (1/23)	Loss: 3301.3284
model saved to models/best.pth
Epoch: 4/100	Progress: 4.35% (1/23)	Loss: 3214.1780
model saved to models/best.pth
Epoch: 5/100	Progress: 4.35% (1/23)	Loss: 3308.2993
model saved to models/best.pth
Epoch: 6/100	Progress: 4.35% (1/23)	Loss: 3172.1448
model saved to models/best.pth
Epoch: 7/100	Progress: 4.35% (1/23)	Loss: 3245.7717
model saved to models/best.pth
Epoch: 8/100	Progress: 4.35% (1/23)	Loss: 3121.5454
model saved to models/best.pth
Epoch: 9/100	Progress: 4.35% (1/23)	Loss: 3230.4919
model saved to models/best.pth
Epoch: 10/100	Progress: 4.35% (1/23)	Loss: 3325.5249
model saved to models/best.pth
Epoch: 11/100	Progress: 4.35% (1/23)	Loss: 3375.3540
model saved to models/best.pth
Epoch: 12/100	Progress: 4.35% (1/23)	Loss: 3041.3201
model saved to models/best.pth
E